# Convnet exercise

In this exercise you will be implementing a (pretty slow) convnet! 

## Preliminaries

In [ ]:
import numpy as np
import scipy.signal
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

## Feedforward network from yesterday's assignment

We extend on the neural network from yesterday which is provided (in full) below. Just execute the cell.

In [ ]:
def onehot(t, num_classes):
    out = np.zeros((t.shape[0], num_classes))
    for row, col in enumerate(t):
        out[row, col] = 1
    return out

def linear(x):
    return x

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def tanh(x):
    return (np.exp(x)-np.exp(-x)) / (np.exp(x)+np.exp(-x))

def relu(x):
    return np.maximum(0, x)

def softplus(x):
    return np.log(np.exp(x) + 1)

class LinearLayer():
    def __init__(self, num_inputs, num_units, scale=0.01):
        self.num_units = num_units
        self.num_inputs = num_inputs
        self.W = np.random.normal(size=(num_inputs, num_units), scale=scale)
        self.b = np.zeros(num_units)

    def __str__(self): 
        return "LinearLayer(%i, %i)" % (self.num_inputs, self.num_units)

    def fprop(self, x, *args):
        self.x = x
        self.a = np.dot(x, self.W) + self.b
        return self.a
        
    def bprop(self, delta_in):
        x_t = np.transpose(self.x)
        self.grad_W = np.dot(x_t, delta_in)
        self.grad_b = delta_in.sum(axis=0)
        W_T = np.transpose(self.W)
        self.delta_out = np.dot(delta_in,W_T)
        return self.delta_out
        
    def update_params(self, lr):
        self.W = self.W - self.grad_W*lr
        self.b = self.b - self.grad_b*lr
        
class SigmoidActivationLayer():
    def __str__(self): 
        return "Sigmoid()"
    
    def fprop(self, x, train=True):
        self.a = 1.0 / (1+np.exp(-x))
        return self.a
        
    def bprop(self, delta_in):
        delta_out = self.a * (1 - self.a)*delta_in
        return delta_out
        
    def update_params(self, lr):
        pass
    
class TanhActivationLayer():
    def __str__(self): 
        return "Tanh()"
    
    def fprop(self, x, train=True):
        self.a = (np.exp(x)-np.exp(-x)) / (np.exp(x)+np.exp(-x))
        return self.a

    def bprop(self, delta_in):
        delta_out = (1 - ((np.exp(self.a)-np.exp(-self.a)) / (np.exp(self.a)+np.exp(-self.a))))*delta_in
        return delta_out
        
    def update_params(self, lr):
        pass

class ReluActivationLayer():
    def __str__(self): 
        return "ReLU()"

    def fprop(self, x, train=True):
        self.a = np.maximum(0, x)
        return self.a
        
    def bprop(self, delta_in):
        return delta_in * (self.a > 0).astype(self.a.dtype)
        
    def update_params(self, lr):
        pass
    
class SoftplusActivationLayer():
    def __str__(self):
        return "Softplus()"
    
    def fprop(self, x, train=True):
        self.a = np.log(np.exp(x) + 1)
        return self.a
    
    def bprop(self, delta_in):
        return delta_in * (1./(1.+np.exp(-x)))
        
    def update_params(self, lr):
        pass

    
class SoftmaxActivationLayer():
    def __str__(self): 
        return "Softmax()"
    
    def fprop(self, x, train=True):
        x_exp = np.exp(x)
        normalizer = x_exp.sum(axis=-1, keepdims=True)
        self.a = x_exp / normalizer
        return self.a
        
    def bprop(self, delta_in):
        return delta_in
        
    def update_params(self, lr):
        pass

class MeanSquaredLoss():
    def __str__(self): 
        return "MeanSquaredLoss()"
    
    def fprop(self, x, t):
        num_batches = x.shape[0]
        cost = 0.5 * (x-t)**2 / num_batches
        return np.mean(np.sum(cost, axis=-1))
        
    def bprop(self, y, t):
        num_batches = y.shape[0]
        delta_out = (1./num_batches) * (y-t)
        return delta_out
        
    def update_params(self):
        pass

class CrossEntropyLoss():
    def __str__(self): 
        return "CrossEntropyLoss()"
    
    def fprop(self, x, t):
        tol = 1e-8
        return np.mean(np.sum(-t * np.log(x + tol), axis=-1))
        
    def bprop(self, y, t):
        num_batches = y.shape[0]
        delta_out = (1./num_batches) * (y-t)
        return delta_out
        
    def update_params(self):
        pass

## Load dataset

In [ ]:
data = np.load('mnist.npz')
num_classes = 10
x_train = data['X_train']
targets_train = data['y_train']
x_train = np.reshape(x_train, (-1, 1, 28, 28))
targets_train = onehot(targets_train, num_classes)

mean = np.mean(x_train)
std = np.std(x_train)
x_train -= mean
x_train /= std

## Gradient checking

In order to verify the correctness of your layers, you will need to [check their gradients numerically](http://ufldl.stanford.edu/wiki/index.php/Gradient_checking_and_advanced_optimization). Below, we have implemented gradient checking functionality for you. Just execute the cell.

In [ ]:
def gradclose(a, b, rtol=None, atol=None):
    rtol = 1e-05 if rtol is None else rtol
    atol = 1e-08 if atol is None else atol
    diff = abs(a - b) - atol - rtol * (abs(a) + abs(b))
    is_close = np.all(diff < 0)
    if not is_close:
        denom = abs(a) + abs(b)
        mask = denom == 0
        rel_error = abs(a - b) / (denom + mask)
        rel_error[mask] = 0
        rel_error = np.max(rel_error)
        abs_error = np.max(abs(a - b))
        print('rel_error=%.4e, abs_error=%.4e, rtol=%.2e, atol=%.2e'
              % (rel_error, abs_error, rtol, atol))
    return is_close



def approx_fprime(x, f, eps=None, *args):
    '''
    Central difference approximation of the gradient of a scalar function.
    '''
    if eps is None:
        eps = np.sqrt(np.finfo(np.float_).eps)
    grad = np.zeros_like(x)
    step = np.zeros_like(x)
    for idx in np.ndindex(x.shape):
        step[idx] = eps * max(abs(x[idx]), 1.0)
        grad[idx] = (f(*((x+step,) + args)) -
                     f(*((x-step,) + args))) / (2*step[idx])
        step[idx] = 0.0
    return grad


def check_grad(layer, x0, seed=1, eps=None, rtol=None, atol=None):
    '''
    Numerical gradient checking of layer bprop.
    '''
    # Check input gradient
    def fun(x):
        y = layer.fprop(x)
        return np.sum(y)

    def fun_grad(x):
        y = layer.fprop(x)
        y_grad = np.ones_like(y)
        x_grad = layer.bprop(y_grad)
        return x_grad

    g_approx = approx_fprime(x0, fun, eps)
    g_true = fun_grad(x0)
    if not gradclose(g_true, g_approx, rtol, atol):
        raise RuntimeError(
            'Incorrect input gradient: \nbprop:\n%s\napprox:\n%s'
            % (g_true, g_approx)
        )

    # Check parameter gradients
    def fun(x, p_idx):
        param_array = layer.params()[p_idx]
        param_array *= 0
        param_array += x
        y = layer.fprop(x0)
        return np.sum(y)

    def fun_grad(x, p_idx):
        param_array = layer.params()[p_idx]
        param_array *= 0
        param_array += x
        out = layer.fprop(x0)
        y_grad = np.ones_like(out)
        layer.bprop(y_grad)
        param_grad = layer.grads()[p_idx]
        return param_grad

    for p_idx, p in enumerate(layer.params()):
        x = np.copy(layer.params()[p_idx])
        g_true = fun_grad(x, p_idx)
        g_approx = approx_fprime(x, fun, eps, p_idx)
        if not gradclose(g_true, g_approx, rtol, atol):
            raise RuntimeError(
                'Incorrect parameter gradient: \nbprop:\n%s\napprox:\n%s'
                % (g_true, g_approx)
            )

## Task #1: Convolution layer


You should implement a 2D convolution layer by filling out the missing pieces and execute the cell. If the gradient check fails, you will get an error.

### Bonus task:
- Implement support for border modes `'full'` and `'valid'`

In [ ]:
def conv_bc01(imgs, filters, padding):
    batch_size, n_channels_img, img_h, img_w = imgs.shape
    n_filters, n_channels, win_h, win_w = filters.shape
    pad_y, pad_x = padding
    if n_channels != n_channels_img:
        raise ValueError('Mismatch in # of channels')

    # Create output array
    out_h = (img_h - win_h + 2*pad_y) + 1
    out_w = (img_w - win_w + 2*pad_x) + 1
    out_shape = (batch_size, n_filters, out_h, out_w)
    out = np.zeros(out_shape)

    # Pad input images
    imgs = np.pad(imgs, ((0, 0), (0, 0), padding, padding), mode='constant')

    # Perform convolution
    for b in range(batch_size):
        for f in range(n_filters):
            for c in range(n_channels):
                out[b, f] += scipy.signal.convolve(imgs[b, c], filters[f, c], mode='valid')
    return out
    

class ConvLayer():
    def __init__(self, n_channels, n_filters, filter_size=5, scale=0.01,
                 border_mode='same'):
        self.n_channels = n_channels
        self.n_filters = n_filters
        self.filter_size = filter_size
        w_shape = (n_filters, n_channels, filter_size, filter_size)
        self.W = np.random.normal(size=w_shape, scale=scale)
        self.b = np.zeros((1, n_filters, 1, 1))
        if border_mode == 'valid':
            self.padding = 0
        elif border_mode == 'same':
            self.padding = filter_size // 2
        elif border_mode == 'full':
            self.padding = filter_size - 1
        else:
            raise ValueError('Invalid border_mode: %s' % border_mode)
        self.padding = (self.padding, self.padding)

        
    def __str__(self): 
        return ("ConvLayer(%i, %i, %i)"
                % (self.n_channels, self.n_filters, self.filter_size))

    def fprop(self, x, *args):
        '''
        Input:
            x: Array of shape (batch_size, n_channels, img_height, img_width)
        Output:
            Array of shape (batch_size, n_filters, out_height, out_width)
        '''
        # Store x for brop()
        self.x = x

        # Perform convolution
        y = conv_bc01(x, self.W, self.padding)
        
        # Add bias
        y = y + self.b
        return y
        
    def bprop(self, dy):
        # Flip weights
        w = self.W[:, :, ::-1, ::-1]
        # Transpose channel/filter dimensions of weights
        w = np.transpose(w, (1, 0, 2, 3))

        # Propagate gradients to x
        dx = conv_bc01(dy, w, self.padding)
        
        # Propagate gradients to weights
        x = np.pad(self.x, ((0, 0), (0, 0), self.padding, self.padding), mode='constant')

        self.grad_W = np.zeros_like(self.W)
        for b in range(dy.shape[0]):
            for f in range(self.W.shape[0]):
                for c in range(self.W.shape[1]):
                    self.grad_W[f, c] += scipy.signal.convolve(x[b, c], dy[b, f], mode='valid')
        self.grad_W = self.grad_W[:, :, ::-1, ::-1]

        # Propagate gradients to bias
        self.grad_b = np.sum(dy, keepdims=True, axis=(0, 2, 3))
        return dx
        
    def update_params(self, lr):
        self.W = self.W - self.grad_W*lr
        self.b = self.b - self.grad_b*lr

    def params(self):
        return self.W, self.b

    def grads(self):
        return self.grad_W, self.grad_b


# Remember to try different parameters. The given parameters are chosen 
# as simple as possible and you may easily discover mistakes in your
# code by changing the parameters.

batch_size = 2
n_channels = 1
img_shape = (5, 5)
n_filters = 2
filter_size = 3

# Border_modes 'full' and 'valid' are left as a bonus task.
border_mode = 'same'

x = np.random.normal(size=(batch_size, n_channels) + img_shape)
layer = ConvLayer(n_channels=n_channels, n_filters=n_filters,
                  filter_size=filter_size, border_mode=border_mode)

check_grad(layer, x)
print('Gradient check passed')

## Task #2: Pooling layer


You should implement average pooling by fillling out the missing pieces and execute the cell. If the gradient check fails, you will get an error.

### Bonus task:
- Implement max pooling.

In [ ]:
class PoolLayer():
    def __init__(self, win_size=3, stride=2):
        self.win_size = win_size
        self.stride = stride
        self.padding = self.win_size // 2

    def __str__(self): 
        return "PoolLayer(%i, %i)" % (self.win_size, self.stride)

    def fprop(self, imgs, *args):
        '''
        Input:
            x: Array of shape (batch_size, n_channels, img_height, img_width)
        Output:
            Array of shape (batch_size, n_channels, out_height, out_width)
        '''
        batch_size, n_channels, img_h, img_w = imgs.shape

        # Store x for brop()
        self.imgs = imgs

        # Create output array
        out_h = (img_h - self.win_size + 2*self.padding) // self.stride + 1
        out_w = (img_w - self.win_size + 2*self.padding) // self.stride + 1
        out = np.zeros((batch_size, n_channels, out_h, out_w))
        
        # Perform average pooling
        imgs = imgs / self.win_size**2
        for b in range(batch_size):
            for c in range(n_channels):
                for y in range(out_h):
                    y_ = y * self.stride
                    for x in range(out_w):
                        x_ = x * self.stride
                        win = imgs[b, c, max(y_, 0):y_+self.win_size,
                                   max(x_, 0):x_+self.win_size]
                        out[b, c, y, x] = np.sum(win)
        return out
        
    def bprop(self, dy):
        dx = np.zeros_like(self.imgs)
        dy = dy / self.win_size**2
        for i in range(dx.shape[0]):
            for c in range(dx.shape[1]):
                for y in range(dy.shape[2]):
                    y_ = y * self.stride
                    for x in range(dy.shape[3]):
                        x_ = x * self.stride
                        dx[i, c, y_:y_+self.win_size, x_:x_+self.win_size] += dy[i, c, y, x]
        return dx

    def update_params(self, lr):
        pass

    def params(self):
        return []

    def grads(self):
        return []

# Remember to try different parameters. The given parameters are chosen 
# as simple as possible and you may easily discover mistakes in your
# code by changing the parameters.

batch_size = 1
n_channels = 1
img_shape = (5, 5)
win_size = 3

x = np.random.normal(size=(batch_size, n_channels) + img_shape)

layer = PoolLayer(win_size=3, stride=2)
check_grad(layer, x)
print('Gradient check passed')

## Task #3: Flatten layer


You should implement flattening such that your convnet layers can be used with a multi-layer perceptron network. Fill out the missing pieces. Gradient checking shouldn't be necessary for this task.

In [ ]:
class FlattenLayer():
    def __str__(self): 
        return "Flatten()"

    def fprop(self, x, *args):
        '''
        Input:
            x: Array of shape (batch_size, n_channels, img_height, img_width)
        Output:
            Array of shape (batch_size, n_channels * img_height * img_width)
        '''

        # Store shape for brop()
        self.shape = x.shape
        y = np.reshape(x, (x.shape[0], -1))
        return y

    def bprop(self, delta_in):
        return np.reshape(delta_in, self.shape)

    def update_params(self, lr):
        pass

## Task #4: A pretty lousy convnet!

Unfortunately, your implementation is too slow to be useful. However, as a final check of your convnet layers, you should try to train a small convnet on MNIST images.

Run the code and verify that you get an accuracy above 0.2 after 150 gradient updates.

In [ ]:
num_samples, n_channels, img_h, img_w = x_train.shape
num_hidden_units = 64
num_classes = 10

layers = [
    ConvLayer(n_channels=1, n_filters=4, filter_size=5, scale=0.1),
    PoolLayer(win_size=3, stride=2),
    ReluActivationLayer(),
    ConvLayer(n_channels=4, n_filters=16, filter_size=5, scale=0.1),
    PoolLayer(win_size=3, stride=2),
    ReluActivationLayer(),
    FlattenLayer(),
    LinearLayer(784, num_hidden_units, scale=0.1),
    ReluActivationLayer(),
    LinearLayer(num_hidden_units, num_classes, scale=0.1),
    SoftmaxActivationLayer(),
]

LossLayer = CrossEntropyLoss()

def forward(x):
    for layer in layers:
        x = layer.fprop(x)
    return x

def backward(y_probs, targets):
    d = LossLayer.bprop(y_probs, targets)
    for layer in reversed(layers):
        d = layer.bprop(d)
    
def update(learning_rate):
    for layer in layers:
        layer.update_params(learning_rate)


from confusionmatrix import ConfusionMatrix
batch_size = 4
num_epochs = 50
learning_rate = 0.05
num_samples = x_train.shape[0]
num_batches = num_samples // batch_size


n_updates = 0
for epoch in range(num_epochs):
    confusion = ConfusionMatrix(num_classes)
    for i in range(num_batches):
        n_updates += 1
        idx = range(i*batch_size, (i+1)*batch_size)
        x_batch = x_train[idx]
        target_batch = targets_train[idx]
        y_probs = forward(x_batch)
        loss = LossLayer.fprop(y_probs, target_batch)
        backward(y_probs, target_batch)
        update(learning_rate)
        confusion.batch_add(target_batch.argmax(-1), y_probs.argmax(-1))
        
        if n_updates % 25 == 0:
            curr_acc = confusion.accuracy()
            print "Update %i : Loss %f Train acc %f" % (n_updates, loss, curr_acc)